In [36]:
import pandas as pd
import requests
import re
import numpy as np
import ast
import time

### We need to get the list of all hashes first

In [22]:
with open('address.txt', 'r') as file:
    address = file.read()

In [23]:
address_to_get_all = address + '?platform=ethereum&limit=100&page='

In [21]:
# we have the maximum of 18 pages, if the limit set to 100 apps per page

In [51]:
data = pd.DataFrame(columns=['category', 'categories', 'created', 'name', 'rank', 'slug', 'teaser'])

In [52]:
from tqdm import tqdm

In [53]:
for page in tqdm(range(1, 19)):
    test = requests.get(f'{address_to_get_all}{page}')
    for i, item in enumerate(test.json()['items']):
        category = test.json()['items'][i]['categories']
        if len(category)>1:
            categories = ' '.join(category)
            category = category[0]
        else:
            categories = 0
            category = category[0]
        created = test.json()['items'][i]['created']
        name = test.json()['items'][i]['name']
        rank = test.json()['items'][i]['rank']
        slug = test.json()['items'][i]['slug']
        teaser = test.json()['items'][i]['teaser']
        df_temp = pd.DataFrame(data={'category': category, 
                                    'categories': categories,
                                    'created': created,
                                    'name': name,
                                    'rank': rank,
                                    'slug': slug,
                                    'teaser': teaser}, index=[0])
        data = pd.concat([data, df_temp])
    time.sleep(0.25)

100%|██████████████████████████████████████████| 18/18 [00:25<00:00,  1.43s/it]


In [59]:
data = data.drop('categories', axis=1)

In [60]:
data.to_csv('dapps_names.csv', index=False)

### FOr each dapp, here or each slug, retrieve a list of hashes

In [62]:
address_for_dapp = f'{address}/'

In [66]:
data.reset_index(drop=True, inplace=True)

In [129]:
hashes = pd.DataFrame(columns=['slug', 'hash', 'description'])

In [130]:
for i, slug in tqdm(enumerate(data.slug)):
    slug = slug
    test = requests.get(f'{address_for_dapp}{data.slug[i]}')
    c = test.json()['item']['contractsMainnet']
    if len(c)==0:
        description = test.json()['item']['description']
        contract_hash = 0
        df_temp = pd.DataFrame(data={'slug': slug,
                                    'hash': contract_hash,
                                    'description': description}, index=[0])
    elif len(c)==1: 
        description = test.json()['item']['description']
        contract_hash = c[0]
        df_temp = pd.DataFrame(data={'slug': slug,
                                    'hash': contract_hash,
                                    'description': description}, index=[0])
    elif len(c)> 1: 
        df_temp = pd.DataFrame(columns=['slug', 'hash', 'description'])
        for contract in c:
            description = test.json()['item']['description']
            contract_hash = contract
            df_t = pd.DataFrame(data={'slug': slug,
                                        'hash': contract_hash,
                                        'description': description}, index=[0])
            df_temp = pd.concat([df_temp, df_t])
    hashes = pd.concat([hashes, df_temp])
    time.sleep(0.25)


0it [00:00, ?it/s]
1it [00:00,  2.03it/s]
2it [00:00,  2.20it/s]
3it [00:01,  2.26it/s]
4it [00:01,  2.40it/s]
5it [00:02,  2.47it/s]
6it [00:02,  2.38it/s]
7it [00:02,  2.43it/s]
8it [00:03,  2.45it/s]
9it [00:03,  2.44it/s]
10it [00:04,  2.47it/s]
11it [00:04,  2.33it/s]
12it [00:04,  2.32it/s]
13it [00:05,  2.42it/s]
14it [00:05,  2.42it/s]
15it [00:06,  2.46it/s]
16it [00:06,  2.50it/s]
17it [00:06,  2.55it/s]
18it [00:07,  2.51it/s]
19it [00:07,  2.55it/s]
20it [00:08,  2.51it/s]
21it [00:08,  2.48it/s]
22it [00:08,  2.52it/s]
23it [00:09,  2.51it/s]
24it [00:09,  2.52it/s]
25it [00:10,  2.55it/s]
26it [00:10,  2.50it/s]
27it [00:10,  2.54it/s]
28it [00:11,  2.49it/s]
29it [00:11,  2.45it/s]
30it [00:12,  2.42it/s]
31it [00:12,  2.45it/s]
32it [00:12,  2.52it/s]
33it [00:13,  2.58it/s]
34it [00:13,  2.60it/s]
35it [00:14,  2.64it/s]
36it [00:14,  2.53it/s]
37it [00:14,  2.63it/s]
38it [00:15,  2.62it/s]
39it [00:15,  2.65it/s]
40it [00:15,  2.65it/s]
41it [00:16,  2.65it/s]
42it 

In [134]:
hashes.reset_index(drop=True, inplace=True)

In [135]:
hashes.to_csv('hashes.csv', index=False)

In [138]:
dapps_df = hashes.merge(data, how='left', left_on='slug', right_on='slug')

In [141]:
dapps_df.to_csv('dapps.csv', index=False)